In [3]:
# ==========================
# train_model.py
# ==========================

import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# ==========================
# 1. Load dataset
# ==========================
df = pd.read_csv("C:\\Projects\\Hackathon\\data\\weatherHistory.csv")  # replace with your dataset

# Ensure datetime
df['Formatted Date'] = pd.to_datetime(df['Formatted Date'])
df = df.sort_values('Formatted Date')

# ==========================
# 2. Fix duplicates
# ==========================
# Option 1: aggregate (recommended)
df = df.groupby('Formatted Date').mean(numeric_only=True)

# ==========================
# 3. Reindex to hourly freq
# ==========================
df = df.asfreq('H')  # enforce hourly timestamps
df['is_original'] = df['thunderstorm'].notna()

# ==========================
# 4. Features & Target
# ==========================
# Target: thunderstorm next hour
df['thunderstorm_next_hour'] = df['thunderstorm'].shift(-1)

# Drop last row (no label for future)
df = df.dropna(subset=['thunderstorm_next_hour'])

# Fill missing values
df = df.interpolate(limit=2)

# Define features (adjust as per your dataset)
features = ['Temperature (C)', 'Humidity', 'Wind Speed (km/h)', 'Pressure (millibars)']
X = df[features]
y = df['thunderstorm_next_hour']

# ==========================
# 5. Train/Test Split
# ==========================
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# ==========================
# 6. Model Training
# ==========================
model = RandomForestClassifier(n_estimators=200, random_state=42)
model.fit(X_train, y_train)

print("Train Accuracy:", model.score(X_train, y_train))
print("Test Accuracy:", model.score(X_test, y_test))

# ==========================
# 7. Save Model
# ==========================
joblib.dump(model, "thunderstorm_model.pkl")


C:\Users\BHAVYA\AppData\Local\Temp\ipykernel_10160\1412110843.py:17: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['Formatted Date'] = pd.to_datetime(df['Formatted Date'])
C:\Users\BHAVYA\AppData\Local\Temp\ipykernel_10160\1412110843.py:29: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df = df.asfreq('H')  # enforce hourly timestamps


KeyError: 'thunderstorm'